<div class="alert alert-block alert-info"><b>Stand-alone ANN version with per-epoch statistics: Experimental</b></div>
Can be used for running overfitting tests or for general testing of new configurations.

In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import load_model

from itertools import cycle
from tensorflow.keras import backend as K_backend
from tensorflow.keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

from tensorflow.keras import Input
from tensorflow.keras import Model

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # Disable GPU if necessary

In [ ]:
def noisy_duplicate_keras_b(data_array_loc): # alt version, might fix the memory issue
    data_array_new = data_array_loc.copy()
    # Element-by-element
    for i in range(trainset_length):
        for j in range(input_num_units):
            data_array_new[i][j] = data_array_new[i][j]*(1 + noise_factor*(0.5 - np.random.random_sample()))
    return data_array_new

def hp_config_print(hp_list):
    label_list = ["Config: ", "/", "/", " - ", "/", "/", ", B ", ", E/LR ", "/", ", EF/NF ", "/"]
    for item, label in zip(hp_list, label_list): print(label, item, sep='', end='')
    print()

def trainset_generator(x_array, y_array, batch_size):
    pairs = zip(x_array, y_array)
    cycle_pairs = cycle(pairs)
    while (True):
        x_batch = []
        y_batch = []
        for _ in range(batch_size):
            x, y = next(cycle_pairs)
            x_batch.append(x)
            y_batch.append(y)
        yield np.array(x_batch), np.array(y_batch)

def met_R2(y_true, y_pred):
    SS_res = K_backend.sum(K_backend.square(y_true - y_pred))
    SS_tot = K_backend.sum(K_backend.square(y_true - K_backend.mean(y_true)))
    return (1 - SS_res/(SS_tot + K_backend.epsilon()))

In [ ]:
# Init
if __name__ == '__main__':
    print("Initialising...")
    work_path = os.path.join("k:" + os.sep, "Archive", "Studying", "NeuralNets", "Python", "Jupyter - Phycocyanin ANN")
    model_type = 1 # Sequential, Function API with one output layer, Function API with three output layers, extras
    generator_enabled = True
    low_priority_enabled = False
    pickle_enabled = False
    flag = 0
    manual_valset_enabled = False
    # Static parameters
    dataset_length = 490 # Out of 490 total
    train_list_length = dataset_length
    input_num_units = 12 #
    output_num_units = 3 # 3 is all
    valset_length = 50 # 50
    # Manually selected test dataset of 50 elements
    if manual_valset_enabled:
        test_points_list = [ 22,  28,  32,  41,  51,  55,  68,  73,  85,  91,
                             96,  97, 109, 119, 124, 137, 150, 166, 173, 181,
                            188, 193, 204, 214, 222, 241, 250, 259, 267, 280,
                            292, 311, 319, 333, 342, 350, 361, 372, 378, 385,
                            390, 394, 401, 408, 415, 435, 450, 461, 473, 482
                           ]
    else: test_points_list = np.random.randint(low=0, high=490, size=50).tolist()
    # Hyperparameters
    # Article hp_config: [10, 8, 9, 'sigmoid', 'sigmoid', 'sigmoid', 10, 5000, 0.001, 4, 0.05]
    hyperparameters = [25, 25, 10, 'relu', 'sigmoid', 'sigmoid', 40, 10000, 0.001, 4, 0.05]
    # Set RNG
    seed = 128
    #
    print("Generator is " + ("enabled" if generator_enabled else "disabled"))
    print("Low process priority is " + ("enabled" if low_priority_enabled else "disabled"))
    print("pickle dumping is " + ("enabled" if pickle_enabled else "disabled"))
    print("Manual valset is " + ("enabled" if manual_valset_enabled else "disabled"))
    print("Model type is:", model_type)
    hp_config_print(hyperparameters)
# End of __main__ section

In [ ]:
# Handling dataset stuff
if __name__ == '__main__':
    if low_priority_enabled:
        import psutil
        this_process = psutil.Process(os.getpid())
        this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    if pickle_enabled:
        import pickle
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Reading data
    df = pd.read_csv(os.path.join(work_path, "Exp data SE 7 (good-504).csv"))
    #df.info()
    df.drop(columns=['No', 'Init OD680', 'OD680'], inplace=True) # Drop the ID and 680s
    df = df.replace(to_replace='???', value=np.NaN) # Replace '???'s with true NaNs for ease of processing
    # Replace NaNs with means
    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = df[i].astype(np.float64) # Convert to float64 explicitly since there were string objects there
        df[i].fillna(df[i].mean(),inplace=True)
    mean_val = (df[df.columns.to_list()].mean()).to_numpy() # Not needed, testing only
    max_val = (df[df.columns.to_list()].max()) # Divide wont' work with a np_array of max values
    for i in df.columns:
        df[i] = df[i].div(max_val[i]) # Divide one by one because if not, the columns will get sorted alphabetically
    max_val = max_val.to_numpy()
    # Valset
    if True: # Are we doing the manual selected test set to imitate the main program behavior?
        test_dataset = (df.loc[test_points_list]).to_numpy()
        df.drop(test_points_list, axis=0, inplace=True)
        train_list_length = dataset_length - valset_length
    # Now, make it into a numpy array
    dataset = df.to_numpy()
    del df
    #print("Max values are:\n", np.around(max_val, decimals=5))
    #print("Mean values are:\n", np.around(mean_val, decimals=5))
    #print("Processed dataset example:\n", dataset[55])
    print("Dataset shape is:", dataset.shape)
    #
    hidden1_num_units, hidden2_num_units, hidden3_num_units, hidden1_func, hidden2_func, hidden3_func, batch_size, epochs, learning_rate, extension_factor, noise_factor = hyperparameters
    # Data normalization for the noise
    dataset_norm = (dataset.copy())/(1 + noise_factor)  # Legacy
    # Randomising data selection: grab the whole set and shuffle it
    # I can't use Keras shuffle and validation split because of the inflation algorithm
    # This way test set is different every time and randomly selected while still being X pre-selected numbers
    trainset_length = train_list_length
    train_in_array = np.zeros((trainset_length, input_num_units))
    train_out_array = np.zeros((trainset_length, output_num_units))
    #
    val_in_array = np.zeros((valset_length, input_num_units))
    val_out_array = np.zeros((valset_length, output_num_units))
    #
    #rand_list = np.random.choice(trainset_length, trainset_length, False)
    for i in range(trainset_length):
        line_to_add = dataset_norm[i]
        train_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
        train_out_array[i] = (line_to_add[input_num_units:].copy())
    for i in range(valset_length):
        line_to_add = test_dataset[i]
        val_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
        val_out_array[i] = (line_to_add[input_num_units:].copy())
    # Extending dataset with 'noisy' duplicates
    train_in_array_temp = train_in_array.copy()
    train_out_array_temp = train_out_array.copy()
    for i in range(0, extension_factor):
        train_in_array_temp = np.concatenate((train_in_array_temp, noisy_duplicate_keras_b(train_in_array)))  # Noisy inputs
        train_out_array_temp = np.concatenate((train_out_array_temp, train_out_array))                      # Clean corresponding outputs
    train_in_array = train_in_array_temp.copy()    # Copy because I'm clearing it, which is probably unnecessary
    train_out_array = train_out_array_temp.copy()
    del train_in_array_temp
    del train_out_array_temp
    #
    #train_list_length = train_list_length*(1 + extension_factor)   # Legacy
    #if len(train_in_array) != train_list_length: print("--- Train list length is off! ---")
    # train_list = dataset[:dataset_length]
    # test_list = dataset[dataset_length:]
    print("Train input shape is:", train_in_array.shape, "train output shape is:", train_out_array.shape)
    print("Val input shape is:", val_in_array.shape, "val output shape is:", val_out_array.shape)
# End of __main__ section  

In [ ]:
# Creating a model for the set of hyperparameters to be used for a single stat loop
if __name__ == '__main__':
    if model_type == 0:
        model = Sequential([
            Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation=hidden1_func),
            Dense(units=hidden2_num_units, activation=hidden2_func),
            Dense(units=hidden3_num_units, activation=hidden3_func),
            Dense(units=output_num_units, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=[met_R2])
    elif model_type == 1:
        inputs = Input(shape=(input_num_units,))
        h1 = Dense(units=hidden1_num_units, activation=hidden1_func)(inputs)
        h2 = Dense(units=hidden2_num_units, activation=hidden2_func)(h1)
        h3 = Dense(units=hidden3_num_units, activation=hidden3_func)(h2)
        outputs = Dense(name='All', units=3, activation="linear")(h3)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(learning_rate=learning_rate), 
            loss={'All': 'mse'},
            metrics={'All': met_R2}
        )
    elif model_type == 2:
        inputs = Input(shape=(input_num_units,))
        h1 = Dense(units=hidden1_num_units, activation=hidden1_func)(inputs)
        h2 = Dense(units=hidden2_num_units, activation=hidden2_func)(h1)
        h3 = Dense(units=hidden3_num_units, activation=hidden3_func)(h2)
        outputs = [Dense(name='OD', units=1, activation="linear")(h3), Dense(name='pH', units=1, activation="linear")(h3), Dense(name='mrel', units=1, activation="linear")(h3)]
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(learning_rate=learning_rate), 
            loss={'OD': 'mse', 'pH': 'mse', 'mrel': 'mse'},
            metrics={'OD': met_R2, 'pH': met_R2, 'mrel': met_R2}
        )
    elif model_type == 3:
        inputs = Input(shape=(input_num_units,))
        h1 = Dense(units=hidden1_num_units, activation=hidden1_func)(inputs)
        outputs = Dense(name='All', units=3, activation="linear")(h1)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(learning_rate=learning_rate), 
            loss={'All': 'mse'},
            metrics={'All': met_R2}
        )
    elif model_type == 4:
        inputs = Input(shape=(input_num_units,))
        h1 = Dense(units=hidden1_num_units, activation=hidden1_func)(inputs)
        h2 = Dense(units=hidden2_num_units, activation=hidden2_func)(h1)
        outputs = Dense(name='All', units=3, activation="linear")(h2)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(learning_rate=learning_rate), 
            loss={'All': 'mse'},
            metrics={'All': met_R2}
        )
    model.summary() # Debug    
# End of __main__ section

In [ ]:
# Calling the model to train
if __name__ == '__main__':
    # Fit: suffle=False because it is already shuffled because of how the dataset is split between train and test
    print("Doing fit...")
    if generator_enabled:
        trainset = trainset_generator(train_in_array, train_out_array, batch_size)
        fit_history = model.fit(
            trainset, validation_data=(val_in_array, val_out_array), steps_per_epoch=len(train_in_array)//batch_size, 
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0), EarlyStopping(monitor='val_loss', mode='min', patience=1000)]
        )
    else:
        if model_type in (2,):
            train_out_array = [np.transpose(train_out_array)[0], np.transpose(train_out_array)[1], np.transpose(train_out_array)[2]]
            val_out_array = [np.transpose(val_out_array)[0], np.transpose(val_out_array)[1], np.transpose(val_out_array)[2]]
        fit_history = model.fit(
            x=train_in_array, y=train_out_array, validation_data=(val_in_array, val_out_array), 
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0), EarlyStopping(monitor='val_loss', mode='min', patience=1000)]
        )
    eval_results = model.evaluate(val_in_array, val_out_array, verbose=0)
    print("Done!")
    if model_type in (0, 1, 3): print("Loss:", "{:.3f}".format(eval_results[0]), "\nMetrics:", "{:.3f}".format(eval_results[1]))
    elif model_type in (2,): print("Loss:", "{:.3f}, {:.3f}, {:.3f}".format(*eval_results[:3]), "\nMetrics:", "{:.3f}, {:.3f}, {:.3f}".format(*eval_results[3:]))    
    # No need to save the model here
    # Now, grab the history to plot a learning curve
    history_dict = fit_history.history
    if pickle_enabled: # Outdated: I have to use pickle dump because the core dies if I try to plot with everythign else loaded
        f = open(os.path.join(work_path, "History_dump_temp", 'wb'))
        pickle.dump(history_dict, f, 2)
        f.close
        f = open(os.path.join(work_path, "Epochs_dump_temp", 'wb'))
        pickle.dump(epochs, f, 2)
        f.close
        print("Dump done!")
# End of __main__ section

In [ ]:
# Plotter functions
def plot_fit_history(x_arr, y_fit, y_val, y_lim='F', met='Metrics', lang=0):
    plt.figure(figsize=(16,10), dpi=100)
    plt.plot(x_arr, y_fit, 'b-', label=("Training data", "Обучающие данные")[lang])
    plt.plot(x_arr, y_val, 'g-', label=("Validation data", "Валидационные данные")[lang])
    #plt.title(("Learning curves: ", "Кривые обучения: ")[lang] + met, fontsize=24)
    plt.xlabel(("Epochs", "Эпохи")[lang], fontsize=20)
    plt.ylabel(met, fontsize=20)
    plt.legend(fontsize=16)
    plt.grid()
    if y_lim != 'F': plt.ylim(y_lim)
    plt.show()

def smooth_fit_history(value_arr, epochs, coeff):
    smooth_arr = list()
    for i in range(0, epochs, coeff): # Currently cuts off the 'tail'
        smooth_arr.append(sum(value_arr[i:i+coeff])/coeff)
    return smooth_arr

In [ ]:
# Plotter for learning curves
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    if not 'pickle_enabled' in locals(): import pickle # If the var is not defined, the previous cell didn't run in this kernel, and as such we need to load from pickle
    if pickle_enabled: 
        f = open(os.path.join(work_path, "History_dump_temp", 'rb'))
        history_dict = pickle.load(f)
        f.close
        f = open(os.path.join(work_path, "Epochs_dump_temp", 'rb'))
        epochs = pickle.load(f)
        f.close
    lang = 1 # English, Russian
    graph_types = (('loss', ('MSE', "Среднеквадратичная ошибка")[lang], [0.0, 0.03]), ('met_R2', ('R2', "Коэффициент детерминации")[lang], [0.5, 1.0]))
    sc = 10 # Smoothing coefficient
    x_arr = list(range(0, epochs, sc))
    if model_type in (0, 1, 3):
        for item in graph_types:
            plot_fit_history(x_arr, smooth_fit_history(history_dict[item[0]], epochs, sc), smooth_fit_history(history_dict['val_' + item[0]], epochs, sc), 
                             item[2], item[1], lang)
    elif model_type in (2,):
        history_avg = {'loss': [], 'met_R2': [], 'val_loss': [], 'val_met_R2': []}
        for i in range(epochs): # Averaging across the outputs to get a similar kind of graph to models 0 and 1
            for key in history_avg:
                vh = 'val_' if 'val' in key else ''
                kh = key[4:] if 'val' in key else key
                history_avg[key].append((history_dict[vh + 'OD_' + kh][i] + history_dict[vh + 'pH_' + kh][i] + history_dict[vh + 'mrel_' + kh][i])/3)
        print("Averaged across outputs graphs:")
        for item in graph_types:
            plot_fit_history(x_arr, smooth_fit_history(history_avg[item[0]], epochs, sc), smooth_fit_history(history_avg['val_' + item[0]], epochs, sc), 
                             item[2], item[1], lang)
        print("Individual output graphs:")
        for parameter in ('OD', 'pH', 'mrel'):
            for item in graph_types:
                key, label = (parameter + '_' + item[0], parameter + ': ' + item[1])
                plot_fit_history(x_arr, smooth_fit_history(history_dict[key], epochs, sc), smooth_fit_history(history_dict['val_' + key], epochs, sc), 
                                 item[2], label, lang)
# End of __main__ section